# Trabalho 2 - IA
Nomes: Diego Severo e Gabrielle Marchioro

Objetivo: Prever se o resultado das partidas com base no dataaset encontrado através do link https://www.kaggle.com/schiller/cartolafc/data.



In [2]:
import pandas as pd
import numpy as np

## Importação do Dataset

In [4]:
pontuacao = pd.read_csv('pontuacao.csv')
posicoes = pd.read_csv('posicoes.csv')
status = pd.read_csv('status.csv')

#atletas = pd.read_csv('2016_atletas.csv')
#scouts = pd.read_csv('2016_scouts.csv')

# Importação dos clubes no período 2014-2017
clubes_2014 = pd.read_csv('2014_clubes.csv')
clubes_2015 = pd.read_csv('2015_clubes.csv')
clubes_2016 = pd.read_csv('2016_clubes.csv')
clubes_2017 = pd.read_csv('2017_clubes.csv')

# Importação das partidas no período 2014-2017
partidas_2014 = pd.read_csv('2014_partidas.csv')
partidas_2015 = pd.read_csv('2015_partidas.csv')
partidas_2016 = pd.read_csv('2016_partidas.csv')
partidas_2017 = pd.read_csv('2017_partidas.csv')


## Pré-processamento

Filtrar as informações para trabalharmos somento com os dados necessários para realizar as previsões.

Primeiramente, será necessário integrar as tuas tabelas de cada ano em apenas uma tabela por ano.

Vamos testar apenas para o ano de 2016 

In [48]:
info_2016 = partidas_2016.set_index('clube_casa_id').join(clubes_2016.set_index('id'), rsuffix = '_casa')
info_2016 = info_2016.set_index('clube_visitante_id').join(clubes_2016.set_index('id'), rsuffix = '_visitante')
info_2016 = info_2016[['rodada','nome','placar_oficial_mandante','nome_visitante','placar_oficial_visitante']]
info_2016['ANO'] = '2016'
info_2016['RESULTADO'] = ''
info_2016.rename(columns = {'nome': 'MANDANTE','rodada': 'RODADA','placar_oficial_mandante': 'GOLS_MANDANTE','nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)



In [69]:
Resultado = {"Vitória_Mandante": 0, "Empate": 1, "Vitória_Visitante": 2}

In [49]:
info_2016

,RODADA,MANDANTE,GOLS_MANDANTE,VISITANTE,GOLS_VISITANTE,ANO,RESULTADO
262,15,Botafogo,3.0,Flamengo,3.0,2016,
262,13,Corinthians,4.0,Flamengo,0.0,2016,
262,30,Fluminense,1.0,Flamengo,2.0,2016,
262,25,Palmeiras,1.0,Flamengo,1.0,2016,
262,28,São Paulo,0.0,Flamengo,0.0,2016,
262,18,Santos,0.0,Flamengo,0.0,2016,
262,33,Atlético-MG,2.0,Flamengo,2.0,2016,
262,8,Cruzeiro,0.0,Flamengo,1.0,2016,
262,2,Grêmio,1.0,Flamengo,0.0,2016,
262,31,Internacional,2.0,Flamengo,1.0,2016,


In [40]:
#nova_coluna = pd.DataFrame({'RESULTADO':['MANDANTE','RODADA','GOLS_MANDANTE','VISITANTE','GOLS_VISITANTE','ANO']})
#info_2016 = info_2016.merge(nova_coluna, left_index = True, right_index = True)

In [64]:
info_2016['RESULTADO'] = np.where(info_2016['GOLS_MANDANTE']>info_2016['GOLS_VISITANTE'],'Vitória_Mandante','Vitória_Visitante')


In [65]:
info_2016

,RODADA,MANDANTE,GOLS_MANDANTE,VISITANTE,GOLS_VISITANTE,ANO,RESULTADO
262,15,Botafogo,3.0,Flamengo,3.0,2016,Vitória_Visitante
262,13,Corinthians,4.0,Flamengo,0.0,2016,Vitória_Mandante
262,30,Fluminense,1.0,Flamengo,2.0,2016,Vitória_Visitante
262,25,Palmeiras,1.0,Flamengo,1.0,2016,Vitória_Visitante
262,28,São Paulo,0.0,Flamengo,0.0,2016,Vitória_Visitante
262,18,Santos,0.0,Flamengo,0.0,2016,Vitória_Visitante
262,33,Atlético-MG,2.0,Flamengo,2.0,2016,Vitória_Visitante
262,8,Cruzeiro,0.0,Flamengo,1.0,2016,Vitória_Visitante
262,2,Grêmio,1.0,Flamengo,0.0,2016,Vitória_Mandante
262,31,Internacional,2.0,Flamengo,1.0,2016,Vitória_Mandante


In [66]:
info_2016.to_csv('dataset.csv')

OBS: Após realizar os condicionais acima para saber os mandantes que venceram seus jogos, defini que qualquer outro resultado seja vitória do visitante e salvei em um novo arquivo csv. Peguei este arquivo e incluí o resultado de "Empate" quando ocorreu empate utilizando o Excel. Após isto eu importei este arquivo para continuar o trabalho aqui :) Vlw flw

In [99]:
info_2016 = pd.read_csv('dataset.csv')

In [100]:
info_2016

,Unnamed: 0,RODADA,MANDANTE,GOLS_MANDANTE,VISITANTE,GOLS_VISITANTE,ANO,RESULTADO
0,284,17,América-MG,0.0,Grêmio,0.0,2016,Empate
1,263,27,América-MG,1.0,Botafogo,0.0,2016,Vitória_Mandante
2,276,33,América-MG,1.0,São Paulo,0.0,2016,Vitória_Mandante
3,277,19,América-MG,1.0,Santos,0.0,2016,Vitória_Mandante
4,285,26,América-MG,1.0,Internacional,0.0,2016,Vitória_Mandante
5,293,32,América-MG,1.0,Atlético-PR,0.0,2016,Vitória_Mandante
6,316,6,América-MG,1.0,Figueirense,0.0,2016,Vitória_Mandante
7,262,35,América-MG,0.0,Flamengo,1.0,2016,Vitória_Visitante
8,266,1,América-MG,0.0,Fluminense,1.0,2016,Vitória_Visitante
9,282,11,América-MG,0.0,Atlético-MG,1.0,2016,Vitória_Visitante


In [101]:
coluna_resultado = info_2016["RESULTADO"].apply(Resultado.get).astype(int)

In [102]:
#vetor_coluna_resultado = coluna_resultado.values
resultado_target = coluna_resultado.values
resultado_target

array([1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 2, 1, 1, 0,
       0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2, 1, 0, 2,
       1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1,
       0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 2, 2,
       1, 1, 0, 2, 2, 1, 1, 1, 1, 0, 0, 0, 2, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1,
       1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 2, 1, 1, 0, 0,
       2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 2, 0, 2, 1, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 2, 2, 2, 1, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 1,
       2, 2,

Feito isto, obtemos o nosso "target" dos resultados das partidas de 2016.

In [111]:
# del(data['RESULTADO'])
# del(data['MANDANTE'])
# del(data['VISITANTE'])

In [114]:
data_train = data.values
data_train

array([[  2.84000000e+02,   1.70000000e+01,   0.00000000e+00,
          0.00000000e+00,   2.01600000e+03],
       [  2.63000000e+02,   2.70000000e+01,   1.00000000e+00,
          0.00000000e+00,   2.01600000e+03],
       [  2.76000000e+02,   3.30000000e+01,   1.00000000e+00,
          0.00000000e+00,   2.01600000e+03],
       ..., 
       [  2.92000000e+02,   1.20000000e+01,   3.00000000e+00,
          2.00000000e+00,   2.01600000e+03],
       [  2.93000000e+02,   3.40000000e+01,   3.00000000e+00,
          2.00000000e+00,   2.01600000e+03],
       [  2.77000000e+02,   1.60000000e+01,   2.00000000e+00,
          3.00000000e+00,   2.01600000e+03]])

Finalizando a fase de Pré-processamento, obtemos o "target" e o "data_train" para podermos fazer a predição dos resultados das partidas

# Treinamento utilizando o K-Neighbors

In [115]:
from sklearn import neighbors

In [135]:
classificador = neighbors.KNeighborsClassifier()
classificador.fit(data_train,resultado_target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [136]:
classificador.predict(data_train)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2,
       0, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2,
       0, 0, 0, 2, 2, 0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 2, 1, 2,
       0, 2,

In [137]:
result_pred = classificador.predict(data_train)

# Avaliação do K-Neighbors

In [120]:
from sklearn import metrics

In [138]:
metrics.accuracy_score(resultado_target,result_pred)

0.65171503957783639

In [139]:
metrics.precision_score(resultado_target,result_pred,average='macro')

0.66627533366287695

In [140]:
metrics.recall_score(resultado_target,result_pred,average='macro')

0.52881803176472475

# Validação Cruzada do K-Neighbors

In [141]:
from sklearn.model_selection import cross_val_score

In [143]:
scores = cross_val_score(classificador,data_train,resultado_target,cv=10,verbose=3,scoring='accuracy')

[CV]  ................................................................
[CV] .................................... , score=0.525, total=   0.0s
[CV]  ................................................................
[CV] ...................................... , score=0.5, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.5897435897435898, total=   0.0s
[CV]  ................................................................
[CV] ...................... , score=0.47368421052631576, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.5405405405405406, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.5405405405405406, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.4594594594594595, total=   0.0s
[CV]  

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [144]:
np.mean(scores)

0.5061400773242879

# Treinamento utilizando Arvore de Decisão

In [145]:
from sklearn import tree

In [146]:
classificador_tree = tree.DecisionTreeClassifier()

In [147]:
classificador_tree = classificador_tree.fit(data_train,resultado_target)

In [149]:
result_pred = classificador_tree.predict(data_train)

In [156]:
result_pred

array([1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 2, 1, 1, 0,
       0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2, 1, 0, 2,
       1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1,
       0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 2, 2,
       1, 1, 0, 2, 2, 1, 1, 1, 1, 0, 0, 0, 2, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1,
       1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 2, 1, 1, 0, 0,
       2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 2, 0, 2, 1, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 2, 2, 2, 1, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 1,
       2, 2,

In [157]:
resultado_target

array([1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2, 2, 1, 1, 0,
       0, 0, 0, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2, 1, 0, 2,
       1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1,
       0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 2, 2,
       1, 1, 0, 2, 2, 1, 1, 1, 1, 0, 0, 0, 2, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1,
       1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 2, 1, 1, 0, 0,
       2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 2, 0, 2, 1, 0, 0,
       0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 2, 2, 2, 1, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 1,
       2, 2,

# Avaliação da Arvore de Decisão

In [150]:
metrics.accuracy_score(resultado_target,result_pred)

1.0

In [151]:
metrics.precision_score(resultado_target,result_pred,average='macro')

1.0

# Validação Cruzada da Árvore de Decisão

In [152]:
scores = cross_val_score(classificador_tree,data_train,resultado_target,cv=10,verbose=3,scoring='accuracy')

[CV]  ................................................................
[CV] .................................... , score=0.975, total=   0.0s
[CV]  ................................................................
[CV] ..................................... , score=0.95, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.9743589743589743, total=   0.0s
[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.0s
[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.0s
[CV]  ................................................................
[CV] ...................................... , score=1.0, total=   0.0s
[CV]  ................................................................
[CV] ........................ , score=0.972972972972973, total=   0.0s
[CV]  

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [153]:
np.mean(scores)

0.98453049203049203